In [28]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [30]:
y = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/train_targets.parquet')
X_estimated= pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/X_train_estimated.parquet')
X_observed= pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/X_train_observed.parquet')
X_test_real = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/X_test_estimated.parquet')
y = y.dropna()

In [31]:
selected_features = ["date_forecast", "direct_rad:W", "clear_sky_rad:W" , "diffuse_rad:W","is_in_shadow:idx", "relative_humidity_1000hPa:p"]

In [32]:
X_estimated_clean = clean_df(X_estimated, selected_features)
X_observed_clean = clean_df(X_observed, selected_features)
X_test_real_clean = clean_df(X_test_real, selected_features)
X_estimated_clean_mean = mean_df(X_estimated_clean)
X_test_real_clean_mean = mean_df(X_test_real_clean)

In [33]:
X_train, X_valid, X_test = training_data_split(X_observed_clean, X_estimated_clean_mean)

In [34]:
X_train, y_train = resize_training_data(X_train, y)
X_valid, y_valid = resize_training_data(X_valid, y)
X_test, y_test = resize_training_data(X_test, y)

In [35]:
"""# Initialize the model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model
rf.fit(X_train, y_train["pv_measurement"])

# Make predictions
predictions = rf.predict(X_test)
predictions = predictions.clip(min = 0, max = None)
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test["pv_measurement"], predictions)

print(f"Mean Absolute Error: {mae}")"""

'# Initialize the model\nrf = RandomForestRegressor(n_estimators=100, random_state=42)\n\n# Fit the model\nrf.fit(X_train, y_train["pv_measurement"])\n\n# Make predictions\npredictions = rf.predict(X_test)\npredictions = predictions.clip(min = 0, max = None)\n# Calculate Mean Absolute Error (MAE)\nmae = mean_absolute_error(y_test["pv_measurement"], predictions)\n\nprint(f"Mean Absolute Error: {mae}")'

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Create the scorer
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Create the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=5)

# Fit the grid search
grid_search.fit(X_train, y_train["pv_measurement"])

# Get the best parameters
best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")

Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}


In [37]:
# Train a new Random Forest model with the best parameters
best_rf_model = RandomForestRegressor(**best_params)
best_rf_model.fit(X_train, y_train["pv_measurement"])

RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                      n_estimators=200)

In [38]:
# Make predictions
predictions = best_rf_model.predict(X_test_real_clean_mean.drop(columns = ["date_forecast"]))
#predictions = best_rf_model.predict(X_test)
#predictions = predictions.clip(min = 0, max = None)
# Calculate the Mean Absolute Error
#mae = mean_absolute_error(y_test["pv_measurement"], predictions)
#print(f"Mean Absolute Error of the best model: {mae}")

In [39]:
np.save('X_test_c.npy', predictions)